In [1]:
import torch
from torch.utils.data import DataLoader 
from torch import nn
from torchvision.datasets import ImageFolder
from torchvision import transforms
from torchvision.models import resnet50, ResNet50_Weights

In [ ]:
# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

# Define model
class NN_model(nn.Module):
    def __init__(self):
        super().__init__()

        self.linear_relu_stack = nn.Sequential(
            # bsx (3x224x224)
            # nn.Conv2d(in_channels=3,out_channels=64, kernel_size=3, stride=3), # 25x224x224
            # nn.MaxPool2d(3), # 25x71x81
            # nn.ReLU(),
            # nn.Conv2d(64, 128, 2), # 50x71x81
            # #nn.MaxPool2d(2), # 50x35x40
            # #nn.ReLU(),
            # nn.Conv2d(128, 300, 3), # 50x71x81
            # nn.MaxPool2d(2), # 50x35x40
            # nn.ReLU(),
            
            # nn.Flatten(),
            
            resnet50(ResNet50_Weights),
            
            nn.Dropout(0.5),
            nn.Linear(1000 , 100),
            nn.ReLU(),
            nn.Linear(100 , 2),

        )

    def forward(self, x):

        logits = self.linear_relu_stack(x)
        return logits
    


Using cuda device


In [3]:
class Dataloader():
  def __init__(self, x, y, bs, shuffle=True):
    self.bs= bs
    #np.arange(len(y)) = np.range(range(len(y)))
    randomizer= np.arange(len(y))
    
    if shuffle:
      np.random.shuffle(randomizer)
      
    self.x= x[randomizer]
    self.y= y[randomizer]
    
  def __len__(self):
    return 4*len(self.y)
  
  def __getitem__(self, idx):
    id_img= idx/4
    rot= 90* idx%4
    return rotation(readimage(id_img),rot)
    # return self.x[idx]/255, self.y[idx]
  
  def __iter__(self):
    return self
  
  def __next__(self):
    pass
    
  def init_dl(x, y, bs):
    ds = np.array[x, y]
    ds.shuffle()
    dl=ds.split(len(x)/bs)
    return dl
  
  


In [4]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    # Set the model to training mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
            
            
def test(dataloader, model, loss_fn):
    # Set the model to evaluation mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    
    # Evaluating the model with torch.no_grad() ensures that no gradients are computed during test mode
    # also serves to reduce unnecessary gradient computations and memory usage for tensors with requires_grad=True
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [ ]:
batch_size =64
epochs =5
lr= 0.0003

def main(batch_size, epochs, lr):    
    
    #Carregar dades
    transform = transforms.Compose([
        transforms.Resize((224, 224)),  # Redimensionar a 224x224p
        transforms.ToTensor(),         # Convertir imatges a tensors de PyTorch
        #transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # Normaliza (valors tipics per a models predeterminats)
    ])
    
    train_dataset = ImageFolder("./data/train", transform=transform)
    test_dataset = ImageFolder("./data/test", transform=transform)
    
    train_dataloader= DataLoader(train_dataset, batch_size, shuffle=True)
    test_dataloader= DataLoader(test_dataset, batch_size,shuffle=True)

    model = NN_model().to(device)
    
    
    loss_fn = nn.CrossEntropyLoss().to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr)
    
    for t in range(epochs):
        print(f"Epoch {t+1}\n-------------------------------")
        train(train_dataloader, model, loss_fn, optimizer)
        test(test_dataloader, model, loss_fn)
    print("Done!")
    
    

main(batch_size, epochs, lr)

/home/iticbcn/Escritorio/CE/IA/Virtualsenv/pyTorch/lib/python3.10/site-packages/torchvision/models/_utils.py:135: UserWarning: Using 'weights' as positional parameter(s) is deprecated since 0.13 and may be removed in the future. Please use keyword parameter(s) instead.
  warnings.warn(
/home/iticbcn/Escritorio/CE/IA/Virtualsenv/pyTorch/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1
-------------------------------
loss: 1.371982  [   64/ 9605]
